In [40]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 23: Safe Cracking ---</h2><p>This is one of the top floors of the nicest tower in EBHQ. The Easter Bunny's private office is here, complete with a safe hidden behind a painting, and who <em>wouldn't</em> hide a star in a safe behind a painting?</p>
<p>The safe has a digital screen and keypad for code entry. A sticky note attached to the safe has a password hint on it: "eggs". The painting is of a large rabbit coloring some eggs. You see <code>7</code>.</p>
<p>When you go to type the code, though, nothing appears on the display; instead, the keypad comes apart in your hands, apparently having been smashed. Behind it is some kind of socket - one that matches a connector in your <a href="11">prototype computer</a>! You pull apart the smashed keypad and extract the logic circuit, plug it into your computer, and plug your computer into the safe.</p>
<p></p>Now, you just need to figure out what output the keypad would have sent to the safe. You extract the <a href="12">assembunny code</a> from the logic chip (your puzzle input).<p></p>
<p>The code looks like it uses <em>almost</em> the same architecture and instruction set that the <a href="12">monorail computer</a> used! You should be able to <em>use the same assembunny interpreter</em> for this as you did there, but with one new instruction:</p>
<p><code>tgl x</code> <em>toggles</em> the instruction <code>x</code> away (pointing at instructions like <code>jnz</code> does: positive means forward; negative means backward):</p>
<ul>
<li>For <em>one-argument</em> instructions, <code>inc</code> becomes <code>dec</code>, and all other one-argument instructions become <code>inc</code>.</li>
<li>For <em>two-argument</em> instructions, <code>jnz</code> becomes <code>cpy</code>, and all other two-instructions become <code>jnz</code>.</li>
<li>The arguments of a toggled instruction are <em>not affected</em>.</li>
<li>If an attempt is made to toggle an instruction outside the program, <em>nothing happens</em>.</li>
<li>If toggling produces an <em>invalid instruction</em> (like <code>cpy 1 2</code>) and an attempt is later made to execute that instruction, <em>skip it instead</em>.</li>
<li>If <code>tgl</code> toggles <em>itself</em> (for example, if <code>a</code> is <code>0</code>, <code>tgl a</code> would target itself and become <code>inc a</code>), the resulting instruction is not executed until the next time it is reached.</li>
</ul>
<p>For example, given this program:</p>
<pre><code>cpy 2 a
tgl a
tgl a
tgl a
cpy 1 a
dec a
dec a
</code></pre>
<ul>
<li><code>cpy 2 a</code> initializes register <code>a</code> to <code>2</code>.</li>
<li>The first <code>tgl a</code> toggles an instruction <code>a</code> (<code>2</code>) away from it, which changes the third <code>tgl a</code> into <code>inc a</code>.</li>
<li>The second <code>tgl a</code> also modifies an instruction <code>2</code> away from it, which changes the <code>cpy 1 a</code> into <code>jnz 1 a</code>.</li>
<li>The fourth line, which is now <code>inc a</code>, increments <code>a</code> to <code>3</code>.</li>
<li>Finally, the fifth line, which is now <code>jnz 1 a</code>, jumps <code>a</code> (<code>3</code>) instructions ahead, skipping the <code>dec a</code> instructions.</li>
</ul>
<p>In this example, the final value in register <code>a</code> is <code>3</code>.</p>
<p>The rest of the electronics seem to place the keypad entry (the number of eggs, <code>7</code>) in register <code>a</code>, run the code, and then send the value left in register <code>a</code> to the safe.</p>
<p><em>What value</em> should be sent to the safe?</p>
</article>


In [41]:
from tabulate import tabulate


def process(
    program: str, register_names: str = "abcd", register_initializer=lambda x: 0
) -> dict[str, int]:
    instructions = [i.split() for i in program.strip().splitlines()]
    registers = {l: register_initializer(l) for l in register_names}
    counter = 0

    while counter < len(instructions):
        instruction = instructions[counter]
        if instruction[0] == "nop":
            counter += 1
        elif instruction[0] == "add":
            _, x, y = instruction
            if y in register_names:
                if x in register_names:
                    registers[y] += registers[x]
                else:
                    registers[y] += int(x)
            counter += 1
        elif instruction[0] == "mul":
            _, x, y = instruction
            if y in register_names:
                if x in register_names:
                    registers[y] *= registers[x]
                else:
                    registers[y] *= int(x)
            counter += 1
        elif instruction[0] == "cpy":
            # cpy x y copies x (either an integer or the value of a register)
            # into register y.
            _, x, y = instruction
            if y in register_names:
                if x in register_names:
                    registers[y] = registers[x]
                else:
                    registers[y] = int(x)
            counter += 1
        elif instruction[0] == "inc":
            # inc x increases the value of register x by one.
            _, x = instruction
            registers[x] += 1
            counter += 1
        elif instruction[0] == "dec":
            # dec x decreases the value of register x by one.
            _, x = instruction
            registers[x] -= 1
            counter += 1
        elif instruction[0] == "jnz":
            # jnz x y jumps to an instruction y away
            # (positive means forward; negative means backward),
            # but only if x is not zero.
            _, x, y = instruction
            y = registers[y] if y in register_names else int(y)
            if x in register_names and registers[x] != 0:
                counter += y
            elif x.isdecimal() and int(x) != 0:
                counter += y
            else:
                counter += 1
        elif instruction[0] == "tgl":
            # tgl x toggles the instruction x away (pointing at instructions
            # - like jnz does: positive means forward; negative means backward):
            _, x = instruction
            if x in register_names:
                v = registers[x]
            else:
                v = int(x)

            if 0 <= counter + v < len(instructions):
                # - If an attempt is made to toggle an instruction outside the program,
                #   nothing happens.

                # - The arguments of a toggled instruction are not affected.
                if len(instructions[counter + v]) == 2:
                    # - For one-argument instructions, inc becomes dec, and all other
                    #   one-argument instructions become inc.
                    # - If tgl toggles itself (for example, if a is 0, tgl a would
                    #   target itself and become inc a), the resulting instruction
                    #   is not executed until the next time it is reached.
                    if instructions[counter + v][0] == "inc":
                        instructions[counter + v][0] = "dec"
                    else:
                        instructions[counter + v][0] = "inc"
                else:
                    # - For two-argument instructions, jnz becomes cpy, and all other
                    #   two-instructions become jnz.
                    if instructions[counter + v][0] == "jnz":
                        instructions[counter + v][0] = "cpy"
                    else:
                        instructions[counter + v][0] = "jnz"
            counter += 1
        else:
            raise ValueError(f"Unkonw isntruction: {instruction}")

    return registers


example = """
cpy 2 a
tgl a
tgl a
tgl a
cpy 1 a
dec a
dec a
"""

result = process(example)
print("Answer Example =", result["a"])

Answer Example = 3


In [42]:
# with open("../input/day23.txt") as f:
#     partI = f.read()

partI = """
cpy a b
dec b
cpy a d
cpy 0 a
cpy b c
inc a
dec c
jnz c -2
dec d
jnz d -5
dec b
cpy b c
cpy c d
dec d
inc c
jnz d -2
tgl c
cpy -16 c
jnz 1 c
cpy 98 c
jnz 86 d
inc a
inc d
jnz d -2
inc c
jnz c -5
"""

result_partI = process(partI, register_initializer=lambda l: 7 if l == "a" else 0)
print("Answer Part I =", result_partI["a"])

Answer Part I = 13468


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>13468</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The safe doesn't open, but it <em>does</em> make several <span title="SUCH INCORRECT! VERY LOCK! WOW!">angry noises</span> to express its frustration.</p>
<p>You're quite sure your logic is working correctly, so the only other thing is... you check the painting again. As it turns out, colored eggs are still eggs. Now you count <code>12</code>.</p>
<p>As you run the program with this new input, the prototype computer begins to <em>overheat</em>. You wonder what's taking so long, and whether the lack of any instruction more powerful than "add one" has anything to do with it. Don't bunnies usually <em>multiply</em>?</p>
<p>Anyway, <em>what value</em> should actually be sent to the safe?</p>
</article>

</main>


In [43]:
# Replace block I with Block II
# nop = no operation so that jumps don't have to be changed
#
# BLOCK I:
# cpy b c
# inc a
# dec c
# jnz c -2
# dec d
# jnz d -5
#
#
#
# Block II:
#
# mul b d
# add d a
# cpy 0 c
# cpy 0 d
# nop
# nop


partII = """
cpy a b
dec b
cpy a d
cpy 0 a
mul b d
add d a
cpy 0 c
cpy 0 d
nop
nop
dec b
cpy b c
cpy c d
dec d
inc c
jnz d -2
tgl c
cpy -16 c
jnz 1 c
cpy 98 c
jnz 86 d
inc a
inc d
jnz d -2
inc c
jnz c -5
"""

result_partII = process(partII, register_initializer=lambda l: 12 if l == "a" else 0)
print("Answer Part II =", result_partII["a"])

Answer Part II = 479010028


<link href="style.css" rel="stylesheet"></link>

<p>Your puzzle answer was <code>479010028</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
